# Import Libraries

In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as wg
import sys
sys.path.append("../src")
import importlib
import support
importlib.reload(support)
from support import readFileDialog, parse_data_file
%matplotlib widget

# Functions

## Read file Vicon

In [23]:
def readFileVicon(file_path, prefix="Victor:", keypoints_mapping_Vicon=None, first_line="Trajectories"):
    if keypoints_mapping_Vicon == None:
        keypoints_mapping_Vicon = ["LANK", "LKNE", "LTHI", "LASI", "LSHO", "LELB", "LWRB", "RANK", "RKNE", "RTHI", "RASI", "RSHO", "RELB", "RWRB"]
    metadata = pd.read_csv(file_path, nrows=1)
    fps = pd.read_csv(file_path, nrows=1)[first_line][0]
    keypoints_mapping_Vicon_tmp = pd.read_csv(file_path, skiprows=[0,1], nrows=1, header=None).values.tolist()[0]
    vicon_data = np.array(pd.read_csv(file_path, skiprows=[0,1,2,4]))
    keypoints_Vicon = np.zeros([len(vicon_data), len(keypoints_mapping_Vicon), 3])
    for i in range(len(keypoints_mapping_Vicon)):
        idx = keypoints_mapping_Vicon_tmp.index(prefix + keypoints_mapping_Vicon[i])
        keypoints_Vicon[:, i, :] = vicon_data[:, idx:idx+3]
    return keypoints_Vicon, keypoints_mapping_Vicon, fps

## Plot 3D Pose

In [41]:
def plot3DPose(keypoints_Vicon, frame_n=0):
    keypoints_Vicon = keypoints_Vicon[frame_n, :, :]
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(keypoints_Vicon[:, 0], keypoints_Vicon[:, 1], keypoints_Vicon[:, 2], marker='o', color='r')
    for i in range(kp_right_n.shape[0]-1):
        ax.plot([keypoints_Vicon[i, 0], keypoints_Vicon[i+1, 0]], [keypoints_Vicon[i, 1], keypoints_Vicon[i+1, 1]],
                zs=[keypoints_Vicon[i, 2], keypoints_Vicon[i+1, 2]], color='black')
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    # Fix aspect ratio
    x, y, z = keypoints_Vicon[:, 0], keypoints_Vicon[:, 1], keypoints_Vicon[:, 2]
    max_range = np.array([x.max()-x.min(), y.max()-y.min(),
                          z.max()-z.min()]).max() / 2.0
    mean_x = x.mean()
    mean_y = y.mean()
    mean_z = z.mean()
    ax.set_xlim(mean_x - max_range, mean_x + max_range)
    ax.set_ylim(mean_y - max_range, mean_y + max_range)
    ax.set_zlim(mean_z - max_range, mean_z + max_range)
    plt.show()

## Interactive 3D Pose

In [44]:
def interactive3DPose(keypoints_Vicon):    
    frame_slider = wg.IntSlider(value=0, min=0, max=len(keypoints_Vicon))
    def interactive3DPlot(frame_n):
        plot3DPose(keypoints_Vicon, frame_n)
    out = wg.interactive_output(interactive3DPlot, {'frame_n': frame_slider})
    display(frame_slider, out)

## Get angles limited

In [140]:
def getAngleLimited(A, B, O):
    try:
        ang = math.degrees(math.atan2(B[1]-O[1], B[0]-O[0]) - math.atan2(A[1]-O[1], A[0]-O[0]))
        if ang < 0:
            ang += 360
        if ang > 180:
            ang = 360 - ang
    except:
        ang = 0
    return ang

## Inverse Kinematics Rowing

In [137]:
def inverseKinematicsRowing(keypoints):
    """ Expected joint order
    if orientation == "Sagittal Right":
        joints_order = ["Right Ankle", "Right Knee", "Right Hip", "Right Shoulder", "Right Elbow", "Right Wrist"]
    else:
        joints_order = ["Left Ankle", "Left Knee", "Left Hip", "Left Shoulder", "Left Elbow", "Left Wrist"]
    """
    angles = np.zeros(5)
    for i in range(5):
        if i == 0:
            O = keypoints[i]
            B = keypoints[i+1]
            C = keypoints[i+2]
            A = np.array([C[0], O[1]])
        else:
            A = keypoints[i-1]
            O = keypoints[i]
            B = keypoints[i+1]
            
        angles[i] = getAngleLimited(A, B, O)
        if i==3:
            if A[0] > B[0]:
                angles[i] = -angles[i]
    return angles

## Get 3D Angle Limited

In [139]:
def get3DAngleLimited(A, O, B):
    f = O-A 
    e = O-B 
    AOVec = np.linalg.norm(f)
    OBVec = np.linalg.norm(e)
    AONorm = f / AOVec
    OBNorm = e / OBVec
    res = AONorm[0] * OBNorm[0] + AONorm[1] * OBNorm[1] + AONorm[2] * OBNorm[2]
    ang = math.degrees(np.arccos(res))
    if ang < 0:
        ang += 360
    if ang > 180:
        ang = 360 - ang
    return ang

## Inverse Kinematics Rowing 3D

In [46]:
def inverseKinematicsRowing3D(keypoints):
    angles_SR = np.zeros(5)
    for i in range(len(angles_SR)):
        if i == 0:
            O_SR= keypoints[i]
            B_SR = keypoints[i+1]
            C_SR = keypoints[i+2]
            A_SR = np.array([C_SR[0], O_SR[1], O_SR[2]])
        else:
            A_SR = keypoints[i-1]
            O_SR = keypoints[i]
            B_SR = keypoints[i+1]      
        angles_SR[i] = get3DAngleLimited(A_SR, O_SR, B_SR)
        if i==3:
            if A_SR[0] > B_SR[0]:
                angles_SR[i] = -angles_SR[i]
    return angles_SR

## Get 3D Angles Vector

In [120]:
def get3DAnglesVector(keypoints_Vicon, keypoints_mapping_Vicon=None):
    if keypoints_mapping_Vicon == None:
            keypoints_mapping_Vicon = ["RANK", "RKNE", "RASI", "RSHO", "RELB", "RWRB"]
    angles_vec = np.zeros([keypoints_Vicon.shape[0], 5])
    for i in range(len(keypoints_Vicon)):
        angles_SR = inverseKinematicsRowing3D(keypoints_Vicon[i])
        angles_vec[i, :] = angles_SR
    return angles_vec

## Get 2D Angles Vector

In [49]:
def get2DAnglesVector(keypoints_Vicon_2D, keypoints_mapping_Vicon=None):
    if keypoints_mapping_Vicon == None:
            keypoints_mapping_Vicon = ["ANK", "KNE", "HIP", "SHO", "ELB", "WRI"]
    angles_vec = np.zeros([len(keypoints_Vicon), len(keypoints_mapping_Vicon)-1])
    for i in range(len(keypoints_Vicon)):
        angles = inverseKinematicsRowing(keypoints_Vicon_2D[i])
        angles_vec[i, :] = angles
    return angles_vec

## Show Rowing Chain Angles Plot

In [50]:
def showRowingChainAnglesPlot(keypoints_xy): 
    """ Expected joint order
    if orientation == "Sagittal Right":
        joints_order = ["Right Ankle", "Right Knee", "Right Hip", "Right Shoulder", "Right Elbow", "Right Wrist"]
    else:
        joints_order = ["Left Ankle", "Left Knee", "Left Hip", "Left Shoulder", "Left Elbow", "Left Wrist"]
    """
    angles = inverseKinematicsRowing(keypoints_xy)
    
    plt.figure()
    for i in range(5):
        O = keypoints_xy[i]
        B = keypoints_xy[i+1]          
        plt.plot([O[0], B[0]], [O[1], B[1]], 'ro-', color = "black")
        circle1 = plt.Circle((O[0], O[1]), 1, color='r')
        plt.text(O[0], O[1], "{}".format(int(round(angles[i]))), bbox=dict(facecolor='red', alpha=0.5))
        plt.gca().set_aspect('equal', adjustable='box')
#     x, y = keypoints_Vicon[:, 0], keypoints_Vicon[:, 1], keypoints_Vicon[:, 2]
#     max_range = np.array([x.max()-x.min(), y.max()-y.min(),
#                           z.max()-z.min()]).max() / 2.0
#     mean_x = x.mean()
#     mean_y = y.mean()
#     mean_z = z.mean()
#     ax.set_xlim(mean_x - max_range, mean_x + max_range)
#     ax.set_ylim(mean_y - max_range, mean_y + max_range)
#     ax.set_zlim(mean_z - max_range, mean_z + max_range)
    plt.grid()
    plt.show()

## Fix Knee Point

In [109]:
def fix_knee_point(kp_vector, kp_mapping):
    kp_mapping_out = kp_mapping.copy()
    kp_vector_out = np.zeros([kp_vector.shape[0], kp_vector.shape[1]-1, kp_vector.shape[2]])
    knee_idx = kp_mapping.index([kp for kp in kp_mapping if "KNE" in kp][0])
    thigh_idx = kp_mapping.index([kp for kp in kp_mapping if "THI" in kp][0])
    kp_mapping_out.remove(kp_mapping[thigh_idx])
    kp_vector_out = np.delete(kp_vector, thigh_idx, axis=1)
    return kp_vector_out, kp_mapping_out

# Open File

Reading Vicon file using the readFileDialog function

In [3]:
file_path = readFileDialog("Open Vicon file")
file_path

'C:/Users/victo/Documents/Motion_analysis/Vicon/Victor_Vicon/Victor Voga24_5min.csv'

In [113]:
keypoints_Vicon, keypoints_mapping_Vicon, fps = readFileVicon(file_path, prefix="Victor:")

In [25]:
keypoints_Vicon.shape

(18976, 14, 3)

In [114]:
kp_right = keypoints_Vicon[:, int(keypoints_Vicon.shape[1]/2):]
kp_right_mapping = keypoints_mapping_Vicon[int(keypoints_Vicon.shape[1]/2):]

In [42]:
frame_n = 500
plot3DPose(kp_right, frame_n)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 3D Analysis

In [178]:
%matplotlib widget
interactive3DPose(kp_right)

IntSlider(value=0, max=18976)

Output()

Fixing the knee position in the keypoint vector

In [117]:
kp_right_fixed, kp_mapping_fixed = fix_knee_point(kp_right, kp_right_mapping)

In [128]:
angles_vec_vicon = get3DAnglesVector(kp_right, kp_right_mapping)

In [129]:
%matplotlib widget
t = np.linspace(0, len(keypoints_Vicon)*(1/fps), len(keypoints_Vicon))
plt.figure()
plt.plot(t, angles_vec_vicon[:, 2])
plt.title("3D Angle Plot")
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 2D Analysis

Assuming a fixed volume origin, we can get the 2D sagittal plane simply by removing the X component of the trajectory, as seen in the 3D trajectory plot. Then, for the analysis we separate left sagittal plane.

In [134]:
kp_right_Vicon_2D = kp_right_fixed[:,:,1:]

In [142]:
frame_n = 500
showRowingChainAnglesPlot(kp_right_Vicon_2D[frame_n])
angles = inverseKinematicsRowing(kp_right_Vicon_2D[frame_n])
print(angles)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 22.37284806 173.24944218 125.86449695 -38.41976533  71.83779104]


In [145]:
angles_vec_2D = get2DAnglesVector(kp_right_Vicon_2D)

In [147]:
angles_vec_2D.shape

(18976, 5)

In [148]:
t = np.linspace(0, len(kp_right_Vicon_2D)*(1/fps), len(kp_right_Vicon_2D))
plt.figure()
plt.plot(t, angles_vec_2D[:, 2])
plt.title("2D Angle Plot")
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 2D vs 3D

In [150]:
%matplotlib widget
joint_n = 2
t = np.linspace(0, len(kp_right_Vicon_2D)*(1/fps), len(kp_right_Vicon_2D))
plt.figure()
plt.plot(t, angles_vec_2D[:, joint_n], label='2D')
plt.plot(t, angles_vec[:, joint_n], label='3D')
plt.title("2D vs 3D Angle Plot")
plt.grid(True)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Comparing Vicon with Openpose

In [156]:
file_path = readFileDialog("Open Data file", "data")
file_path

'C:/Users/victo/Documents/Motion_analysis/Vicon/Victor_Voga24_5min/Victor_Voga24_5minBIK.data'

In [170]:
data, var_names = parse_data_file(file_path)

In [176]:
kp_openpose = data["keypoints"]
md_openpose = data["metadata"]
ang_openpose = data["angles"]
fps_openpose = md_openpose["fps"]